### Made To look up hrrr data
Change startdate and enddate in lines 10 and 11
currently only looks up surface: "SNOD", "GUST", "ASNOW_acc_fcst", "PRES", "TMP"


## V1 
raw data, needs converting
output: historical_hrrr_fcst.csv

In [13]:
import pandas as pd
import numpy as np
import datetime
import s3fs
import numcodecs as ncd
import xarray as xr
import cartopy.crs as ccrs

# define vars
startdate = "2023-10-01"
enddate = "2023-10-02"
point_lat = 39.58148838130895
point_lon = -105.94259648925797
level = 'surface'
fs = s3fs.S3FileSystem(anon=True)
projection = ccrs.LambertConformal(central_longitude=262.5, central_latitude=38.5, 
                                   standard_parallels=(38.5, 38.5),
                                   globe=ccrs.Globe(semimajor_axis=6371229, semiminor_axis=6371229))

x, y = projection.transform_point(point_lon, point_lat, ccrs.PlateCarree())
chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
nearest_point = chunk_index.sel(x=x, y=y, method="nearest")

# Data Retrieval Function
def retrieve_data(s3_url, var):
    with fs.open(s3_url, 'rb') as compressed_data:
        buffer = ncd.blosc.decompress(compressed_data.read())
        dtype = "<f4" if "surface/PRES" in s3_url else "<f2"
        chunk = np.frombuffer(buffer, dtype=dtype)
        data_array = np.reshape(chunk, (len(chunk)//(150*150), 150, 150))
        return data_array

# Data Collection
data_collection = []
variables = ["SNOD", "GUST", "ASNOW_acc_fcst", "PRES", "TMP"]

for day in pd.date_range(start=startdate, end=enddate):
    for hour in range(24):
        date_str = day.strftime('%Y%m%d')
        hour_str = f'{hour:02d}'
        row = {'date': date_str, 'hour': hour_str}
        for var in variables:
            data_url = f'hrrrzarr/sfc/{date_str}/{date_str}_{hour_str}z_fcst.zarr/{level}/{var}/{level}/{var}/'
            fcst_chunk_id = f"0.{nearest_point.chunk_id.values}"
            data = retrieve_data(data_url + fcst_chunk_id, var)
            gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
            row[var] = gridpoint_forecast.mean()
        data_collection.append(row)

# Convert to DataFrame and Export to CSV
df = pd.DataFrame(data_collection)
df.to_csv('historical_hrrr_fcst.csv', index=False)
print(df)

        date hour      SNOD       GUST  ASNOW_acc_fcst          PRES     TMP
0   20231001   00  0.000000  12.359375        0.000000  68498.125000  281.25
1   20231001   01  0.000000  13.039062        0.000038  68506.664062  279.75
2   20231001   02  0.000000  13.046875        0.000000  68497.781250  281.25
3   20231001   03  0.000000  12.304688        0.000000  68506.664062  281.25
4   20231001   04  0.000000  12.507812        0.000000  68508.890625  282.25
5   20231001   05  0.000000  12.085938        0.000000  68499.445312  282.00
6   20231001   06  0.000854  11.398438        0.000948  68454.789062  280.50
7   20231001   07  0.000000  12.187500        0.000000  68507.218750  282.75
8   20231001   08  0.000000  11.804688        0.000000  68496.664062  282.50
9   20231001   09  0.000000  11.117188        0.000000  68523.890625  282.75
10  20231001   10  0.000000  11.484375        0.000000  68519.445312  283.00
11  20231001   11  0.000000  11.031250        0.000000  68505.554688  283.75

## V2: 
Next version converts each variable based on maya's code conversions she used
output: historical_hrrr_fcst_trnsfrm.csv

In [14]:
import pandas as pd
import numpy as np
import datetime
import s3fs
import numcodecs as ncd
import xarray as xr
import cartopy.crs as ccrs

# define vars
startdate = "2023-10-01"
enddate = "2023-10-02"
point_lat = 39.58148838130895
point_lon = -105.94259648925797
level = 'surface'
fs = s3fs.S3FileSystem(anon=True)
projection = ccrs.LambertConformal(central_longitude=262.5, central_latitude=38.5, 
                                   standard_parallels=(38.5, 38.5),
                                   globe=ccrs.Globe(semimajor_axis=6371229, semiminor_axis=6371229))

x, y = projection.transform_point(point_lon, point_lat, ccrs.PlateCarree())
chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
nearest_point = chunk_index.sel(x=x, y=y, method="nearest")

# Data Retrieval Function
def retrieve_data(s3_url, var):
    with fs.open(s3_url, 'rb') as compressed_data:
        buffer = ncd.blosc.decompress(compressed_data.read())
        dtype = "<f4" if "surface/PRES" in s3_url else "<f2"
        chunk = np.frombuffer(buffer, dtype=dtype)
        data_array = np.reshape(chunk, (len(chunk)//(150*150), 150, 150))
        return data_array

# Data Collection
data_collection = []
variables = ["SNOD", "GUST", "ASNOW_acc_fcst", "PRES", "TMP"]

for day in pd.date_range(start=startdate, end=enddate):
    for hour in range(24):
        date_str = day.strftime('%Y%m%d')
        hour_str = f'{hour:02d}'
        row = {'date': date_str, 'hour': hour_str}
        for var in variables:
            data_url = f'hrrrzarr/sfc/{date_str}/{date_str}_{hour_str}z_fcst.zarr/{level}/{var}/{level}/{var}/'
            fcst_chunk_id = f"0.{nearest_point.chunk_id.values}"
            data = retrieve_data(data_url + fcst_chunk_id, var)
            gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
            mean_value = gridpoint_forecast.mean()

            # Apply specific transformations for each variable
            if var == "PRES":
                row[var] = mean_value / 3386
            elif var == "SNOD" or var == "ASNOW_acc_fcst":
                row[var] = mean_value * 39.37
            elif var == "GUST":
                row[var] = mean_value * 2.24
            elif var == "TMP":
                row[var] = (mean_value - 273.15) * (9/5) + 32
            else:
                row[var] = mean_value

        data_collection.append(row)

# Convert to DataFrame and Export to CSV
df = pd.DataFrame(data_collection)
df.to_csv('historical_hrrr_fcst_trnsfrm.csv', index=False)
print(df)


        date hour      SNOD      GUST  ASNOW_acc_fcst       PRES    TMP
0   20231001   00  0.000000  27.68500        0.000000  20.229807  46.58
1   20231001   01  0.000000  29.20750        0.001483  20.232328  43.88
2   20231001   02  0.000000  29.22500        0.000000  20.229705  46.58
3   20231001   03  0.000000  27.56250        0.000000  20.232328  46.58
4   20231001   04  0.000000  28.01750        0.000000  20.232986  48.38
5   20231001   05  0.000000  27.07250        0.000000  20.230196  47.93
6   20231001   06  0.033641  25.53250        0.037321  20.217008  45.23
7   20231001   07  0.000000  27.30000        0.000000  20.232492  49.28
8   20231001   08  0.000000  26.44250        0.000000  20.229375  48.83
9   20231001   09  0.000000  24.90250        0.000000  20.237416  49.28
10  20231001   10  0.000000  25.72500        0.000000  20.236103  49.73
11  20231001   11  0.000000  24.71000        0.000000  20.232001  51.08
12  20231001   12  0.001640  23.31000        0.046370  20.201258

## V3:
cleaning up by taking output and rounds to the hundredth 
output: historical_hrrr_fcst_trnsfrm_rnd.csv

In [15]:
import pandas as pd
import numpy as np
import datetime
import s3fs
import numcodecs as ncd
import xarray as xr
import cartopy.crs as ccrs

# define vars
startdate = "2023-10-01"
enddate = "2023-10-03"
point_lat = 39.58148838130895
point_lon = -105.94259648925797
level = 'surface'
fs = s3fs.S3FileSystem(anon=True)
projection = ccrs.LambertConformal(central_longitude=262.5, central_latitude=38.5, 
                                   standard_parallels=(38.5, 38.5),
                                   globe=ccrs.Globe(semimajor_axis=6371229, semiminor_axis=6371229))

x, y = projection.transform_point(point_lon, point_lat, ccrs.PlateCarree())
chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
nearest_point = chunk_index.sel(x=x, y=y, method="nearest")

# Data Retrieval Function
def retrieve_data(s3_url, var):
    with fs.open(s3_url, 'rb') as compressed_data:
        buffer = ncd.blosc.decompress(compressed_data.read())
        dtype = "<f4" if "surface/PRES" in s3_url else "<f2"
        chunk = np.frombuffer(buffer, dtype=dtype)
        data_array = np.reshape(chunk, (len(chunk)//(150*150), 150, 150))
        return data_array

# Data Collection
data_collection = []
variables = ["SNOD", "GUST", "ASNOW_acc_fcst", "PRES", "TMP"]

for day in pd.date_range(start=startdate, end=enddate):
    for hour in range(24):
        date_str = day.strftime('%Y%m%d')
        hour_str = f'{hour:02d}'
        row = {'date': date_str, 'hour': hour_str}
        for var in variables:
            data_url = f'hrrrzarr/sfc/{date_str}/{date_str}_{hour_str}z_fcst.zarr/{level}/{var}/{level}/{var}/'
            fcst_chunk_id = f"0.{nearest_point.chunk_id.values}"
            data = retrieve_data(data_url + fcst_chunk_id, var)
            gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
            mean_value = gridpoint_forecast.mean()

            # Apply specific transformations for each variable and round to two decimal places
            if var == "PRES":
                row[var] = round(mean_value / 3386, 2)
            elif var == "SNOD" or var == "ASNOW_acc_fcst":
                row[var] = round(mean_value * 39.37, 2)
            elif var == "GUST":
                row[var] = round(mean_value * 2.24, 2)
            elif var == "TMP":
                row[var] = round((mean_value - 273.15) * (9/5) + 32, 2)
            else:
                row[var] = round(mean_value, 2)

        data_collection.append(row)

# Convert to DataFrame and Export to CSV
df = pd.DataFrame(data_collection)
df.to_csv('historical_hrrr_fcst_trnsfrm_rnd.csv', index=False)

# Print the DataFrame
print(df)


        date hour  SNOD   GUST  ASNOW_acc_fcst   PRES    TMP
0   20231001   00   0.0  27.68            0.00  20.23  46.58
1   20231001   01   0.0  29.21            0.00  20.23  43.88
2   20231001   02   0.0  29.22            0.00  20.23  46.58
3   20231001   03   0.0  27.56            0.00  20.23  46.58
4   20231001   04   0.0  28.02            0.00  20.23  48.38
..       ...  ...   ...    ...             ...    ...    ...
67  20231003   19   0.0  12.85            0.00  20.19  32.63
68  20231003   20   0.0  11.69            0.00  20.20  30.83
69  20231003   21   0.0  11.37            0.01  20.20  32.18
70  20231003   22   0.0   9.99            0.00  20.21  32.18
71  20231003   23   0.0   9.19            0.00  20.21  32.18

[72 rows x 7 columns]


to-do: figure out what is most relevant to compare, forecast vs annalyzed or hrrr forecast vs weather station data
